# Merge JEP and JFP images

In [2]:
import os
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
from config import raw_data_dir

from rasterio.plot import show

%matplotlib inline

images_dir = '../data/interim/images'
out_images_dir = '../data/interim/images-merged'

In [3]:
! ls $images_dir

2017-01-01 2017-02-10 2017-03-22 2017-06-20 2017-07-15 2017-08-19
2017-01-31 2017-03-12 2017-05-31 2017-07-10 2017-08-04


In [ ]:
from rasterio.merge import merge

def merge_rasters(date, band):
    
    out_dir = os.path.join(out_images_dir, date)
    out_fp = os.path.join(out_dir, f'{band}.jp2')
    
    if os.path.isfile(out_fp):
        print(out_fp + ' exists. Skipping...')
        return
    
    raster_1 = rasterio.open(os.path.join(images_dir, date, f'{band}_JEP.jp2'))
    raster_2 = rasterio.open(os.path.join(images_dir, date,  f'{band}_JFP.jp2'))
    
    out, transform = merge([raster_1, raster_2], precision=1)
    
    meta = raster_1.meta.copy()
    
    meta.update({
        'height': out.shape[1],
        'width': out.shape[2],
        'transform': transform
    })
    
    raster_1.close()
    raster_2.close()
    
    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
        
    with rasterio.open(out_fp, 'w', **meta) as dst:
        dst.write(out)

In [ ]:
from tqdm import tqdm

bands = ['B01','B02','B03','B04','B05','B06','B07','B08','B8A','B09','B10','B11','B12','TCI']
dates = ['2017-01-01','2017-02-10','2017-03-22','2017-06-20','2017-07-15','2017-08-19','2017-01-31','2017-03-12','2017-05-31','2017-07-10','2017-08-04']

for date in tqdm(dates, desc='dates'):
    for band in bands:
        merge_rasters(date, band)

# Merge resolution groups

In [6]:
import os
import gc

import rasterio
from rasterio.merge import merge
from tqdm import tqdm

from config import interim_data_dir

dates = ['2017-01-01','2017-02-10','2017-03-22','2017-06-20','2017-07-15','2017-08-19','2017-01-31','2017-03-12','2017-05-31','2017-07-10','2017-08-04']


res_groups = {
    "60": ["B01", "B09", "B10"],
    "20": ["B05", "B06", "B07", "B8A", "B11", "B12"],
    "10": [
        "B02",
        "B03",
        "B04",
        "B08",
        #  "TCI"
    ],
}

in_dir = os.path.join(interim_data_dir, "images-merged/")
res_groups_dir = os.path.join(interim_data_dir, "res-groups/")


def merge_res_group(res_group, date):

    out_dir = os.path.join(res_groups_dir, date)
    out_fp = os.path.join(out_dir, f"{res_group}.jp2")

    if os.path.isfile(out_fp):
        print(f'{out_fp} exists. Skipping')
        return

    print(f"Merging rasters of {res_group}m resolution...",end='')

    bands = res_groups[res_group]
    bands_fpaths = [os.path.join(in_dir, date, f"{band}.jp2") for band in bands]

    # Read meta data from first band
    src0 = rasterio.open(bands_fpaths[0])
    meta = src0.meta.copy()
    src0.close()

    # Update meta data
    meta.update({"count": len(bands)})

    if not os.path.isdir(out_dir):
        os.makedirs(out_dir)

    with rasterio.open(out_fp, "w", **meta) as dst:
        for i, fp in enumerate(bands_fpaths, start=1):
            with rasterio.open(fp) as band_raster:
                dst.write_band(i, band_raster.read(1))

    print("done!")
    print("\t-->", out_fp)

    test_saved_data(out_fp, len(bands))


def test_saved_data(fp, expected_bands):
    with rasterio.open(fp) as raster:
        assert raster.count == expected_bands


### Run on single

In [8]:
merge_res_group('20','2017-02-10')

Merging rasters of 20m resolution...done!
	--> /Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-02-10/20.jp2


### Run for all

In [7]:
for date in dates:
    print("-" * 50, date, "-" * 50)
    print("")
    for res_group in res_groups.keys():
        merge_res_group(res_group, date)

-------------------------------------------------- 2017-01-01 --------------------------------------------------

/Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-01-01/60.jp2 exists. Skipping
/Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-01-01/20.jp2 exists. Skipping
/Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-01-01/10.jp2 exists. Skipping
-------------------------------------------------- 2017-02-10 --------------------------------------------------

/Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-02-10/60.jp2 exists. Skipping
/Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-02-10/20.jp2 exists. Skipping
Merging rasters of 10m resolution...done!
	--> /Users/renier.botha/dev/personal/ds/zindi/farm-pin/data/interim/res-groups/2017-02-10/10.jp2
-------------------------------------------------- 2017-03-22 --------------------